In [1]:
import os.path
import threading
import time
import pygame as pg
import sys
import tkinter as tk
from tkinter import filedialog
import pygame.display
from os import path
from scipy.fftpack import dct
from math import pi
import sys, math, wave, numpy , random ,colorsys , librosa

import librosa
import numpy as np
import pandas as pd 
import audioread

import joblib

pygame 2.3.0 (SDL 2.24.2, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np
import os, sys, librosa
from scipy import signal
from scipy import ndimage
from scipy.interpolate import interp1d
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
from matplotlib import gridspec
import IPython.display as ipd
from numba import jit
import pandas as pd
from collections import OrderedDict
import random

sys.path.append('..')
import libfmp.b
import libfmp.c3
import libfmp.c8


In [11]:
key_list = [40,167,297,417]

# Load song file
file_path = filedialog.askopenfilename(title="Select song file", filetypes=(("wav audio files", "*.wav"), ("mp3 audio files", "*.mp3")))
print(file_path)

# Read the selected music file
f = wave.open(file_path, 'rb') 
frequency = f.getframerate()
params = f.getparams() #  nchannels, sampwidth, framerate, nframes, comptype, compname 
nchannels, sampwidth, framerate, nframes = params[:4]
str_data  = f.readframes(nframes) 
wave_data = numpy.frombuffer(str_data, dtype = numpy.short)
wave_data.shape = -1,2
wave_data = wave_data.T

def separate_melody_accompaniment(x, Fs, N, H, traj, n_harmonics=10, tol_cent=50.0):
    # Compute STFT
    X = librosa.stft(x, n_fft=N, hop_length=H, win_length=N, pad_mode='constant')
    Fs_feature = Fs / H
    T_coef = np.arange(X.shape[1]) / Fs_feature
    freq_res = Fs / N
    F_coef = np.arange(X.shape[0]) * freq_res

    # Adjust trajectory
    traj_X_values = interp1d(traj[:, 0], traj[:, 1], kind='nearest', fill_value='extrapolate')(T_coef)
    traj_X = np.hstack((T_coef[:, None], traj_X_values[:, None, ]))

    # Compute binary masks
    mask_mel = libfmp.c8.convert_trajectory_to_mask_cent(traj_X, F_coef, n_harmonics=n_harmonics, tol_cent=tol_cent)
    mask_acc = np.ones(mask_mel.shape) - mask_mel

    # Compute masked STFTs
    X_mel = X * mask_mel
    X_acc = X * mask_acc

    # Reconstruct signals
    x_mel = librosa.istft(X_mel, hop_length=H, win_length=N, window='hann', center=True, length=x.size)
    x_acc = librosa.istft(X_acc, hop_length=H, win_length=N, window='hann', center=True, length=x.size)

    return x_mel, x_acc

def random_discard(onset_times, onset_frames, threshold=1):
    i = 1
    new_onset_times = [onset_times[0]]
    new_onset_frames = [onset_frames[0]]
    while i < len(onset_times):
        diff = onset_times[i] - onset_times[i-1]
        if not diff < threshold or not random.random()>=0.5:
            new_onset_times.append(onset_times[i])
            new_onset_frames.append(onset_frames[i])
        i += 1
    return new_onset_times, new_onset_frames

# generation of beatmap
def generate_beatmap(file_path, melody_extract_flag=False):
    file_name = os.path.splitext(file_path)[0]
    print(file_name)
    if os.path.isfile(file_name + '.txt') and False:
        print(f"The beatmap file {file_path} already exists.")
    else:
        y, sr = librosa.load(file_path)
        duration = librosa.get_duration(y=y, sr=sr)
        print(f'duration: {duration}')
        onset_frames = librosa.onset.onset_detect(y=y, sr=sr, 
                                                  hop_length=100,
                                                  backtrack=False,
                                                  units='samples',
                                                #   pre_max=20,
                                                #   post_max=20,
                                                #   pre_avg=100,
                                                #   post_avg=100,
                                                #   delta=0.2,
                                                #   wait=0
                                                  )
        onset_times = librosa.samples_to_time(onset_frames)
        # onset_times = librosa.frames_to_time(onset_frames)
        onset_times -= 0.6 # time for the note to fall down
        onset_times = [x for x in onset_times if x >= 0]
        notes_num = len(onset_times)
        print(f'number of notes: {notes_num}')

        # calculate average rate
        avg_rate = notes_num/duration
        
        print(avg_rate)

        if avg_rate>3 and melody_extract_flag:
            # perform melody extraction and separation
            # however, this is too slow
            # Compute trajectory
            traj, Z, T_coef, F_coef_hertz, F_coef_cents = libfmp.c8.compute_traj_from_audio(y, Fs=sr, 
                                                    constraint_region=None, gamma=0.1)

            N = 2048
            H = N//4
            x_mel, x_acc = separate_melody_accompaniment(y, Fs=sr, N=N, H=H, traj=traj, n_harmonics=30, tol_cent=50)

            onset_frames = librosa.onset.onset_detect(y=x_mel, sr=sr)
            onset_times = librosa.frames_to_time(onset_frames)
            onset_times -= 0.6 # time for the note to fall down
            onset_times = [x for x in onset_times if x >= 0]
            new_notes_num = len(onset_times)
            print(f'number of notes after melody extraction: {new_notes_num}')
            if new_notes_num > notes_num:
                onset_times = random_discard(onset_times)
                print(f'number of notes after random discard: {len(onset_times)}')
        elif avg_rate>3 and not melody_extract_flag:
            print(onset_times)
            onset_times, onset_frames = random_discard(onset_times, onset_frames)
            print(f'number of notes after random discard: {len(onset_times)}')

        def estimate_pitch(segment, sr, fmin=50.0, fmax=2000.0):
            # Compute autocorrelation of input segment.
            r = librosa.autocorrelate(segment)

            # Define lower and upper limits for the autocorrelation argmax.
            r[:int(sr/fmax)] = 0 # delete all value after upper limits
            r[int(sr/fmin):] = 0 # delete all value before lower limit
            
            # Find the location of maximum autocorrelation
            loc = r.argmax() 

            # Find the location of the maximum autocorrelation and determine the fundamental frequency
            f0 = float(sr/loc)
            return f0
        
        def estimate_pitch_by_onset(x, onset_samples, i, sr):
            n0 = onset_samples[i]
            n1 = onset_samples[i+1]
            f0 = estimate_pitch(x[n0:n1], sr=sr)
            return f0
        
        y_pitch = np.array([
            estimate_pitch_by_onset(y, onset_frames, i, sr=sr)
            for i in range(len(onset_frames)-1)
        ])

        print(f'number of pitch: {len(y_pitch)}')
        print(f'number of onset_times: {len(onset_times)}')
        print(f'number of onset_frames: {len(onset_frames)}')

        min_pitch = np.min(y_pitch)
        max_pitch = np.max(y_pitch)
        sd_pitch = np.std(y_pitch)
        interval_pitch = (max_pitch - min_pitch) / 4.0
        range_pitch = [min_pitch, min_pitch+interval_pitch, min_pitch+interval_pitch*2, min_pitch+interval_pitch*3]
                        
        # export to .txt file
        with open(file_name + '.txt', 'w') as f:
            f.write('\n'.join(['%.4f' % onset_time for onset_time in onset_times]))

        def diff(a,b):
            return abs(a-b)
        
        def between(num, this):
            for temp in range(len(this)-1):
                if num >= this[temp] and num <= this[temp+1]:
                    return temp
            return len(this)-1
        
        with open(file_name + '_note.txt', 'w') as f:
            cur_col_index = between(y_pitch[0], range_pitch)
            ref_pitch = y_pitch[0]
            for i in range(len(onset_times)-1):
                if y_pitch[i] >= ref_pitch:
                    cur_col_index = int(cur_col_index + int(diff(y_pitch[i], ref_pitch)/(sd_pitch/3)))
                else:
                    cur_col_index = int(cur_col_index - int(diff(y_pitch[i], ref_pitch)/(sd_pitch/3)))
                ref_pitch = y_pitch[i]
                if cur_col_index<0 or cur_col_index>3:
                    cur_col_index = between(y_pitch[0], range_pitch)
                print(cur_col_index)
                f.write(f'{str(key_list[cur_col_index])} \n')

        

generate_beatmap(file_path)

/Users/tracywong/Desktop/AIST3110 - Project/Melody Quest/La Campanella.wav
/Users/tracywong/Desktop/AIST3110 - Project/Melody Quest/La Campanella
duration: 317.2426757369615
number of notes: 1401
4.416177605189615
[4.452154195011338, 4.7877551020408164, 5.114285714285715, 5.445351473922903, 5.785487528344672, 6.116553287981859, 6.438548752834468, 6.765079365079365, 7.0961451247165535, 7.454421768707483, 7.799092970521542, 8.143764172335601, 8.48843537414966, 8.8421768707483, 9.20952380952381, 9.57687074829932, 9.989569160997732, 10.511111111111111, 11.867120181405896, 12.025850340136055, 12.175510204081633, 12.32517006802721, 12.456689342403628, 12.592743764172337, 12.742403628117914, 12.878458049886621, 13.055328798185942, 13.20952380952381, 13.345578231292517, 13.481632653061224, 13.617687074829933, 13.76281179138322, 14.062131519274377, 14.220861678004535, 14.361451247165533, 14.520181405895691, 14.642630385487529, 14.814965986394558, 14.955555555555556, 15.241269841269842, 15.39546

In [19]:
this = [1, 2, 3, 4]
num = 4.5

def between(num, this):
    for temp in range(len(this)-1):
        print(temp)
        if num >= this[temp] and num <= this[temp+1]:
            return temp
    return len(this)-1

between(4.5, this)

0
1
2


3

In [35]:
def diff(a,b):
    return abs(a-b)

diff(1,2)

1

In [26]:
y_pitch = generate_beatmap('Suzume.wav')

Suzume
duration: 236.41401360544216
number of notes: 398
1.683487344638686
[ 501.13636364  294.          432.35294118  441.          329.10447761
  262.5         214.0776699   259.41176471  294.          344.53125
  361.47540984  329.10447761  265.6626506  1297.05882353  294.
  432.35294118  355.64516129  334.09090909  282.69230769  222.72727273
  265.6626506   329.10447761  361.47540984  329.10447761  297.97297297
 2004.54545455  148.98648649  525.          432.35294118  432.35294118
  294.          294.          130.47337278  256.39534884  310.56338028
  367.5         334.09090909  282.69230769  147.98657718  294.
  918.75        147.98657718  525.          212.01923077  432.35294118
  294.          294.          131.25        259.41176471  334.09090909
  355.64516129  334.09090909  147.98657718  147.         1002.27272727
  302.05479452  918.75         73.5          73.5          73.7458194
   73.7458194    65.625        65.82089552  882.          918.75
 2004.54545455  711.29032258

In [14]:
def get_color_from_mood_detection(file_path):
    def extract_features(audio_file):
        y, sr = librosa.load(audio_file)
        
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_stft_mean = np.mean(chroma_stft)
        chroma_stft_var = np.var(chroma_stft)
        
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        spectral_centroid_mean = np.mean(spectral_centroid)
        spectral_centroid_var = np.var(spectral_centroid)
        
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        spectral_bandwidth_mean = np.mean(spectral_bandwidth)
        spectral_bandwidth_var = np.var(spectral_bandwidth)
        
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        spectral_contrast_mean = np.mean(spectral_contrast)
        spectral_contrast_var = np.var(spectral_contrast)
        
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
        zero_crossing_rate_mean = np.mean(zero_crossing_rate)
        zero_crossing_rate_var = np.var(zero_crossing_rate)
        
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        mfccs_mean = np.mean(mfccs, axis=1)
        mfccs_var = np.var(mfccs, axis=1)
        
        features = {
            'tempo': tempo.reshape(1,)[0],
            'beat_frames': beat_frames.mean(axis=0),
            'chroma_stft_mean': chroma_stft_mean,
            'chroma_stft_var': chroma_stft_var,
            'zero_crossing_rate_mean': zero_crossing_rate_mean,
            'zero_crossing_rate_var': zero_crossing_rate_var,
            'spectral_centroid_mean': spectral_centroid_mean,
            'spectral_centroid_var': spectral_centroid_var,
            'spectral_bandwidth_mean': spectral_bandwidth_mean,
            'spectral_bandwidth_var': spectral_bandwidth_var,
            'spectral_contrast_mean': spectral_contrast_mean,
            'spectral_contrast_var': spectral_contrast_var,
            'mfcc1_mean': mfccs_mean[0],
            'mfcc1_var': mfccs_var[0],
            'mfcc2_mean': mfccs_mean[1],
            'mfcc2_var': mfccs_var[1],
            'mfcc3_mean': mfccs_mean[2],
            'mfcc3_var': mfccs_var[2],
            'mfcc4_mean': mfccs_mean[3],
            'mfcc4_var': mfccs_var[3],
            'mfcc5_mean': mfccs_mean[4],
            'mfcc5_var': mfccs_var[4],
            'mfcc6_mean': mfccs_mean[5],
            'mfcc6_var': mfccs_var[5],
            'mfcc7_mean': mfccs_mean[6],
            'mfcc7_var': mfccs_var[6],
            'mfcc8_mean': mfccs_mean[7],
            'mfcc8_var': mfccs_var[7],
            'mfcc9_mean': mfccs_mean[8],
            'mfcc9_var': mfccs_var[8],
            'mfcc10_mean': mfccs_mean[9],
            'mfcc10_var': mfccs_var[9],
            'mfcc11_mean': mfccs_mean[10],
            'mfcc11_var': mfccs_var[10],
            'mfcc12_mean': mfccs_mean[11],
            'mfcc12_var': mfccs_var[11],
            'mfcc13_mean': mfccs_mean[12],
            'mfcc13_var': mfccs_var[12],
            'mfcc14_mean': mfccs_mean[13],
            'mfcc14_var': mfccs_mean[13],
            'mfcc15_mean': mfccs_mean[14],
            'mfcc15_var': mfccs_mean[14],
            'mfcc16_mean': mfccs_mean[15],
            'mfcc16_var': mfccs_mean[15],
            'mfcc17_mean': mfccs_mean[16],
            'mfcc17_var': mfccs_mean[16],
            'mfcc18_mean': mfccs_mean[17],
            'mfcc18_var': mfccs_mean[17],
            'mfcc19_mean': mfccs_mean[18],
            'mfcc19_var': mfccs_mean[18],
            'mfcc20_mean': mfccs_mean[19],
            'mfcc20_var': mfccs_mean[19],
        }

        return features


    features = extract_features(file_path)

    features_unseen_song = [value for value in features.values()]
    print(features_unseen_song)

    # Load the model from disk
    loaded_model = joblib.load('LR_arousal.sav')

    # Use the loaded model to make predictions
    arousal_prediction = loaded_model.predict([features_unseen_song])

    print(arousal_prediction)

    # Load the model from disk
    loaded_model = joblib.load('RF_valence.sav')

    # Use the loaded model to make predictions
    valence_prediction = loaded_model.predict([features_unseen_song])

    print(valence_prediction)

    if arousal_prediction > 0 and valence_prediction > 0:
        mood_color = (253, 242, 204) # Yellow
    elif arousal_prediction > 0 and valence_prediction < 0:
        mood_color = (225, 213, 231) # purple 
    elif arousal_prediction < 0 and valence_prediction > 0:
        mood_color = (213, 232, 212) # green
    else: 
        mood_color = (218, 232, 252) # blue

    print(mood_color)

    return mood_color

In [15]:
get_color_from_mood_detection('Suzume.wav')

[78.30255681818181, 4879.727891156463, 0.28731546, 0.08891992, 0.08762854907864369, 0.004318315835356498, 2166.4268133529386, 704592.7987935998, 2412.027549255099, 272795.5613415031, 23.80823350385762, 120.08022596846467, -102.78026, 10346.461, 97.51306, 1074.7656, -9.038109, 486.09705, 5.8457584, 348.1039, -7.580399, 285.15207, 0.2173403, 259.47275, 5.109651, 270.0096, -5.5011277, 166.02768, 1.1370957, 124.80932, 1.3976524, 151.18602, -1.5433738, 123.92385, -1.6164248, 164.22693, -1.3579829, 102.74991, 5.295546, 5.295546, -0.98800665, -0.98800665, 3.9276922, 3.9276922, 2.4419322, 2.4419322, 9.073247, 9.073247, 3.3339946, 3.3339946, 6.2815347, 6.2815347]
[0.69614804]
[0.10092308]
(253, 242, 204)


(253, 242, 204)